# GNNの実装を解読するために

## GNNとはどのようなアーキテクチャなのか？

GNN（Graph Neural Network）は、グラフ構造データを扱うためのニューラルネットワークの一種である。グラフはノード（頂点）とエッジ（辺）から構成される。物質への応用の場合、ノード特徴量としては、元素の種類などをone-hot encodingでベクトルに変換したものを初期値とすることが多い。エッジはノード間の関係性を表しており、物質の場合は「ある距離より近いノード間にはエッジがある」と設定する場合が多い。GNNは、エッジを介して、つながっている先のノードの情報を用いてノードの特徴量を更新していく点に特徴がある。

## PaiNN
PaiNN（Polarizable Interaction Neural Network）は、同変性をもたせたGNNアーキテクチャの一つである。同変性というのが何かというと

$$
Rf(x) = f(Rx)
$$

つまり、入力変数に何らかの作用（ここでは回転とか）をしたものを入力した結果と、もともとの変数を入力した結果に同じ作用をしたものが等しい、という性質である。物質の性質は、物質全体を回転させたり平行移動させたりしても変わらないため、GNNに同変性をもたせることは重要である。PaiNNでは、ノード特徴量としてスカラー量とベクトル量の両方を用いることで同変性を実現している。

ベクトル量の特徴量を同変に保てる演算は内積、定数倍、外積、線形結合である。その一方、ニューラルネットワークで必須となる非線形変換には注意が必要である。
単純にベクトルの要素ごとに非線形変換を掛けてしまうと、同変性が崩れてしまう。そこで、ゲート機構という方法がよく用いられる。
これは、非線形変換（シグモイド関数やReLU, SiLUなど）への入力を、ベクトルの特徴量と別のベクトルとの内積によってスカラーにしたものし、その変換結果を使ってもとのベクトル特徴量をスケーリングするという方法である。こうすることで、ベクトル特徴量自体には非線形変換を直接かけないため、同変性を保つことができる。

さて、PaiNNで行われている特徴量の更新を見ていこう。PaiNNにおける最大の特徴はスカラーとベクトル、両方の特徴量を持っていることである。この２つの特徴量がお互いに影響を受けながら更新をされていく。ベクトル特徴量を持っている最大の優位性は、角度の情報が自然と入ることにある。
スカラー特徴量の典型例はボンドの長さである。しかし、ボンドの長さをいくら集積してもボンドの角度の情報は直接的に得ることはできない。全結合型NNPでは角度情報を明示的に入れ込むような特徴量が用いられるが、角度の計算は$O(N^2)$のコストであるため、避けたい。一方、ボンドの向きに相当するベクトル特徴量を持っておくと、その総和の二乗を取れば角度情報が自然と入ることになる。この演算自体は$O(N)$である。
そこで、特徴量が２系統になるが、どちらも$O(N)$で済む演算を使うことで、効率的かつ構造記述をリッチにするという戦略を取っている。

### スカラー特徴量のmessage passing
PaiNNにおける入力は、原子量$Z_i$と位置ベクトル$\mathbf{r}_i$である。各ノード$i$に対して、スカラー特徴量$\mathbf{s}_i$とベクトル特徴量$\mathbf{v}_i$を持つ。
スカラー特徴量の初期値は$Z_i$の埋め込みから得られる。そのスカラー特徴量の更新につかうメッセージパッシングは以下のように定義される。

$$
\Delta s_i^m =  \sum_{j \in \mathcal{N}(i)} \phi_s(s_j) W_s(r_{ij})
$$

この部分は、ほぼSchNetと一緒である。$r_{ij}$は原子iとｊの間の距離であり、$\phi_s$の実態はニューラルネットワークである。
ここで、$W_s(r_{ij})$は、以下のような原子間距離を展開して得られるベクトルの線形変換によって得られる「不変フィルター」と呼ばれるものである。
$$
e_n=\sin \left(\frac{n\pi}{r_{cut}} r_{ij}\right)/r_{ij}
$$
展開の方法がSchNetとは少し異なる。

展開の部分はコードの以下の部分に相当する

```
def sinc_expansion(edge_dist: torch.Tensor, edge_size: int, cutoff: float):
    """
    calculate sinc radial basis function:
    
    sin(n *pi*d/d_cut)/d
    """
    n = torch.arange(edge_size, device=edge_dist.device) + 1
    return torch.sin(edge_dist.unsqueeze(-1) * n * torch.pi / cutoff) / edge_dist.unsqueeze(-1)
```

### ベクトル特徴量のmessage passing
ベクトル特徴量に対するメッセージパッシングは以下のように行われる

$$
\Delta \mathbf{v}_i^m =  \sum_{j \in \mathcal{N}(i)} \mathbf{v}_j\phi_{vv}(s_j) W_{vv}(r_{ij})
+ \sum_{j \in \mathcal{N}(i)} \phi_{vs}(s_j) W_{vs}(r_{ij}) \frac{\mathbf{r}_{ij}}{r_{ij}}
$$

第一項は前述のゲート機構と同じアイデアで、ベクトルをスケール変換する演算に対応し、第２項は実はフィルター$W(r_{ij})$を微分したようなものとみなすことができる。これによって「同変な」フィルターを実現している。

スカラー特徴量、ベクトル特徴量いずれにおいても
$$
\phi(s_j) W(r_{ij})
$$
という形が共通している。なので、一斉に計算し、最後にその結果を三分割するような実装がされている。

```
class MessageLayer(nn.Module):
    #atomwise message passing
    def __init__(self, natom_basis, n_radial, cutoff):
        super(MessageLayer, self).__init__()
        self.natom_basis=natom_basis
        self.n_radial=n_radial
        self.cutoff=cutoff
        self.interaction_context_network=nn.Sequential(
            nn.Linear(self.natom_basis, natom_basis),
            nn.SiLU(),
            nn.Linear(self.natom_basis, natom_basis*3),
        )

        
        self.filter_network=nn.Sequential(
            nn.Linear(self.n_radial,natom_basis*3),
        )

    def forward(self, q, mu, edge_index, edge_weight):
        #q: scalar representation
        #mu: vector representation

        #edge_weightとしては原子間ベクトルがそのまま入っている
        
        #message passing
        #スカラー特徴量を変換
        x = self.interaction_context_network(q)
        distances=torch.norm(edge_weight, dim=-1)
        #単位ベクトルを取得
        directions=edge_weight/distances.unsqueeze(-1)
        
        #ここで展開をおこなっている
        basis_fn=sinc_expansion(distances, self.n_radial, self.cutoff)
        cutoff=cosine_cutoff(distances, self.cutoff).unsqueeze(-1)
        #カットオフの導入
        filter_Wij=self.filter_network(basis_fn)*cutoff
        
        #ペア情報の取得
        idx_i=edge_index[0]
        idx_j=edge_index[1]

        #相手側のインデックスを取得
        xj=x[idx_j]
        muj=mu[idx_j]

        #psi*Wの計算
        x=filter_Wij*xj

        #psi_s, psi_vv, psi_vsに分割
        dq, dmuR, dmumu=torch.split(x, self.natom_basis, dim=-1)

        #aggregation
        index=idx_i.unsqueeze(1).expand_as(dq)
        
        q_update=torch.zeros_like(q,device=q.device)
        q_update=torch.scatter_add(q_update, 0, index, dq)

        #dmuRはpsi_vsに相当、dmumuはpsi_vvに相当
        dmuR = dmuR.unsqueeze(1)
        dmumu = dmumu.unsqueeze(1)
        dmu = dmuR * directions[..., None] + dmumu * muj

        index=idx_i.unsqueeze(-1).unsqueeze(-1).expand_as(dmu)
        
        mu_update=torch.zeros_like(mu,device=mu.device)
        mu_update = torch.scatter_add(mu_update, 0, index, dmu)
        
        q=q+q_update
        mu=mu+mu_update

        return q, mu
```

interaction_context_networkが$\phi(s_j)$に対応し、filter_networkが$W(r_{ij})$に対応している。

## Update layer
作成したメッセージを用いて、ノードの特徴量を更新する際にもいくつかの工夫が用いられている。更新の際には非線形変換が入るのが通例なので、ここでゲート機構を用いる。ゲート機構には、ベクトル特徴量をスカラーに落とすための内積が用いられている。その内積に使う行列を、ベクトル特徴量に対する線形変換として学習させている。

その際、２つの行列を生成し、それらを組み合わせて使うことで、うまくスカラー特徴量と、ベクトル特徴量を結びつけることができる。
生成される行列２つをそれぞれ$U,V$と表記し、スカラー特徴量、ベクトル特徴量のUpdateを以下のように定義する。

$$
\Delta s_i^u = a_{ss} (s_i, ||V\mathbf{v}_i||) +a_{sv} (s_i, ||V\mathbf{v}_i||) <U\mathbf{v}_i, V\mathbf{v}_i>
$$

$$
\Delta u_i^u = a_{vv} (s_i, ||V\mathbf{v}_i||) U\mathbf{v}_i
$$

ここでもやはり共通した構造$a(s_i, ||V\mathbf{v}_i||)$が出てきており、これの実態は全結合NNなので、まとめて処理して出力を分割するような方法を取る

対応する実装部分は以下のようになっている

```
class UpdateLayer(nn.Module):
    def __init__(self, natom_basis, epsilon):
        """
        updating scaler representation using vector representation
        
        natom_basis: embedding atom type dimension, dimension of scalar representation
        epsilon: small value to avoid zero division
        """
        super(UpdateLayer, self).__init__()
        self.register_buffer("epsilon", torch.tensor(epsilon))
        self.natom_basis=natom_basis
        
        #s_i,Vv_iの結合したものを受け取るので最初の次元がself.natom_basis*2
        self.intraatomic_context_net=nn.Sequential(
            nn.Linear(self.natom_basis*2, self.natom_basis),
            nn.SiLU(),
            nn.Linear(self.natom_basis, self.natom_basis*3),
        )
        self.mu_channel_mix=nn.Sequential( 
            nn.Linear(self.natom_basis, self.natom_basis*2),
        )

    def forward(self, q, mu):
        """
        updating scaler representation using vector representation
        
        q: scalar representation
        mu: vector representation
        """
        mu_mix=self.mu_channel_mix(mu)
        mu_V,mu_W=torch.split(mu_mix, self.natom_basis, dim=-1)
        mu_Vn = torch.sqrt(torch.sum(mu_V**2, dim=-2, keepdim=False) + self.epsilon)
        
        ctx = torch.cat([q, mu_Vn], dim=-1)
        x = self.intraatomic_context_net(ctx)
        dq_intra, dmu_intra, dqmu_intra = torch.split(x, self.natom_basis, dim=-1)

        dmu_intra = dmu_intra.unsqueeze(1) * mu_W

        dqmu_intra = dqmu_intra * torch.sum(mu_V * mu_W, dim=1, keepdim=False)

        q = q + dq_intra + dqmu_intra
        mu = mu + dmu_intra
        
        return q,mu
```

U,Vを作成するのがmu_channel_mixに対応し、intraatomic_context_netが$a(s_i, ||V\mathbf{v}_i||)$に対応している。
ノルムが0になるのを防ぐためにepsilonが加えられている点に注意。